# Adjusting Budget and Revenue for Inflation #

In [60]:
import numpy as np
import pandas as pd
import json

import statsmodels.api as sm

import matplotlib.pyplot as plt 
%matplotlib inline

import random as rand
import seaborn as sns

from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from datetime import datetime

In [59]:
movies = pd.read_csv("cleaned_data.csv")

In [61]:
#Getting release dates
release_dates = movies.iloc[:,[0,14]]
release_dates

#Only Getting Data w/o NaN
release_dates_filt = release_dates[release_dates['release_date'].notnull()]

In [62]:
#Release dates transformed into DateTime format in a Pandas Dataset:

dates = []

for i in range(len(release_dates_filt)):
    #print(type(release_dates.iloc[i][1]))
    entry_date = datetime.strptime(release_dates_filt.iloc[i][1], '%Y-%m-%d').date()
    dates.append([release_dates_filt.iloc[i][0],entry_date])

dates = pd.DataFrame(dates)

In [63]:
years = []
for i in range(len(release_dates_filt)):
    entry_year = dates.iloc[i][1].year
    years.append([release_dates_filt.iloc[i][0],entry_year])
years_arr = np.array(years)
years_arr = years_arr.T
years_df = pd.DataFrame()
years_df["id"] = years_arr[0]
years_df["Year"] = years_arr[1]

In [65]:
# We need inflation data or ticket data from 1915~current
# Everything will be converted into estimated number of tickets sold, then back to 2023 rates 

ticketprices = pd.read_csv("yearly_ticketprice.csv")
#ticketprices.dtypes

In [66]:
final_tx = years_df.join(ticketprices.set_index("Year"),on="Year")

In [68]:
columns = movies.iloc[:,[0,14,15,3]]
columns

,id,release_date,revenue,budget
0,10003,1997-04-03,118063304,68000000
1,100042,2014-11-12,169837010,40000000
2,10012,2005-02-25,19294901,35000000
3,10013,1986-10-05,41382841,18000000
4,10014,1985-10-31,29999213,3000000
...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000
4509,99861,2015-04-22,1405403694,280000000
4510,9988,2006-08-25,19179969,17500000
4511,9989,2001-01-12,18195610,30000000


In [69]:
merged = columns.merge(final_tx, how = "inner", on="id")
merged

,id,release_date,revenue,budget,Year,Ticket Price
0,10003,1997-04-03,118063304,68000000,1997,4.59
1,100042,2014-11-12,169837010,40000000,2014,8.17
2,10012,2005-02-25,19294901,35000000,2005,6.41
3,10013,1986-10-05,41382841,18000000,1986,3.71
4,10014,1985-10-31,29999213,3000000,1985,3.55
...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55
4509,99861,2015-04-22,1405403694,280000000,2015,8.43
4510,9988,2006-08-25,19179969,17500000,2006,6.55
4511,9989,2001-01-12,18195610,30000000,2001,5.66


In [70]:
merged["ROI"] = (merged["revenue"]-merged["budget"])/(merged["budget"])
merged

,id,release_date,revenue,budget,Year,Ticket Price,ROI
0,10003,1997-04-03,118063304,68000000,1997,4.59,0.736225
1,100042,2014-11-12,169837010,40000000,2014,8.17,3.245925
2,10012,2005-02-25,19294901,35000000,2005,6.41,-0.448717
3,10013,1986-10-05,41382841,18000000,1986,3.71,1.299047
4,10014,1985-10-31,29999213,3000000,1985,3.55,8.999738
...,...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55,0.694118
4509,99861,2015-04-22,1405403694,280000000,2015,8.43,4.019299
4510,9988,2006-08-25,19179969,17500000,2006,6.55,0.095998
4511,9989,2001-01-12,18195610,30000000,2001,5.66,-0.393480


In [71]:
merged["Tickets Sold"] = merged["revenue"]/merged["Ticket Price"]
merged

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold
0,10003,1997-04-03,118063304,68000000,1997,4.59,0.736225,2.572185e+07
1,100042,2014-11-12,169837010,40000000,2014,8.17,3.245925,2.078788e+07
2,10012,2005-02-25,19294901,35000000,2005,6.41,-0.448717,3.010125e+06
3,10013,1986-10-05,41382841,18000000,1986,3.71,1.299047,1.115440e+07
4,10014,1985-10-31,29999213,3000000,1985,3.55,8.999738,8.450483e+06
...,...,...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55,0.694118,2.198473e+07
4509,99861,2015-04-22,1405403694,280000000,2015,8.43,4.019299,1.667146e+08
4510,9988,2006-08-25,19179969,17500000,2006,6.55,0.095998,2.928240e+06
4511,9989,2001-01-12,18195610,30000000,2001,5.66,-0.393480,3.214772e+06


In [72]:
merged["Revenue in 2023"] = merged["Tickets Sold"]*10.45
merged

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold,Revenue in 2023
0,10003,1997-04-03,118063304,68000000,1997,4.59,0.736225,2.572185e+07,2.687934e+08
1,100042,2014-11-12,169837010,40000000,2014,8.17,3.245925,2.078788e+07,2.172334e+08
2,10012,2005-02-25,19294901,35000000,2005,6.41,-0.448717,3.010125e+06,3.145581e+07
3,10013,1986-10-05,41382841,18000000,1986,3.71,1.299047,1.115440e+07,1.165635e+08
4,10014,1985-10-31,29999213,3000000,1985,3.55,8.999738,8.450483e+06,8.830754e+07
...,...,...,...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55,0.694118,2.198473e+07,2.297405e+08
4509,99861,2015-04-22,1405403694,280000000,2015,8.43,4.019299,1.667146e+08,1.742167e+09
4510,9988,2006-08-25,19179969,17500000,2006,6.55,0.095998,2.928240e+06,3.060010e+07
4511,9989,2001-01-12,18195610,30000000,2001,5.66,-0.393480,3.214772e+06,3.359437e+07


In [73]:
merged

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold,Revenue in 2023
0,10003,1997-04-03,118063304,68000000,1997,4.59,0.736225,2.572185e+07,2.687934e+08
1,100042,2014-11-12,169837010,40000000,2014,8.17,3.245925,2.078788e+07,2.172334e+08
2,10012,2005-02-25,19294901,35000000,2005,6.41,-0.448717,3.010125e+06,3.145581e+07
3,10013,1986-10-05,41382841,18000000,1986,3.71,1.299047,1.115440e+07,1.165635e+08
4,10014,1985-10-31,29999213,3000000,1985,3.55,8.999738,8.450483e+06,8.830754e+07
...,...,...,...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55,0.694118,2.198473e+07,2.297405e+08
4509,99861,2015-04-22,1405403694,280000000,2015,8.43,4.019299,1.667146e+08,1.742167e+09
4510,9988,2006-08-25,19179969,17500000,2006,6.55,0.095998,2.928240e+06,3.060010e+07
4511,9989,2001-01-12,18195610,30000000,2001,5.66,-0.393480,3.214772e+06,3.359437e+07


In [75]:
merged1 = movies.merge(final, how = "outer", on=["id","revenue","budget","release_date"])
merged1.columns

Index(['id', 'adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords', 'Year',
       'Ticket Price', 'ROI', 'Tickets Sold', 'Revenue in 2023'],
      dtype='object')

In [76]:
months = []
for i in range(len(release_dates_filt)):
    entry_month = dates.iloc[i][1].month
    months.append([release_dates_filt.iloc[i][0],entry_month])

months_arr = np.array(months)
months_arr = months_arr.T

months_df = pd.DataFrame()
months_df["id"] = months_arr[0]
months_df["Month"] = months_arr[1]

In [77]:
merged1.columns

Index(['id', 'adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords', 'Year',
       'Ticket Price', 'ROI', 'Tickets Sold', 'Revenue in 2023'],
      dtype='object')

In [78]:
merged2 = merged1.merge(months_df, how = "inner", on="id")
#avoid duplications in budget, revenue, release_date
merged2.columns

Index(['id', 'adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords', 'Year',
       'Ticket Price', 'ROI', 'Tickets Sold', 'Revenue in 2023', 'Month'],
      dtype='object')

In [83]:
merged2.to_csv('movies_inflation_adjusted.csv',index=False)

In [84]:
movies_infl = pd.read_csv("movies_inflation_adjusted.csv")
movies_infl.iloc[0]

id                                                                   10003
adult                                                                False
belongs_to_collection    {'id': 86224, 'name': 'The Saint Collection', ...
budget                                                            68000000
genres                   [{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...
homepage                                                               NaN
imdb_id                                                          tt0120053
original_language                                                       en
original_title                                                   The Saint
overview                 Ivan Tretiak, Russian Mafia boss who wants to ...
popularity                                                        10.97633
poster_path                               /uA24D8JCg21RDINMZ3vFaQYdIzW.jpg
production_companies     [{'name': 'Paramount Pictures', 'id': 4}, {'na...
production_countries     